In [13]:
import os
import cv2
import numpy as np

In [14]:
def dark_channel(image, window_size=15):
    padded_image = cv2.copyMakeBorder(image, window_size // 2, window_size // 2, window_size // 2, window_size // 2, cv2.BORDER_CONSTANT)
    dark_channel = np.min(padded_image, axis=2)
    return dark_channel[window_size // 2 : -window_size // 2, window_size // 2 : -window_size // 2]

In [15]:
def atmospheric_light(image, dark_channel, percentile=0.001):
    flat_dark_channel = dark_channel.flatten()
    num_pixels = flat_dark_channel.shape[0]
    
    # Determine the number of pixels to select
    num_pixels_to_select = max(1, int(num_pixels * percentile))
    
    # If the number of pixels to select exceeds the total number of pixels, select all pixels
    if num_pixels_to_select >= num_pixels:
        num_pixels_to_select = num_pixels
    
    indices = np.argpartition(flat_dark_channel, -num_pixels_to_select)[-num_pixels_to_select:]
    atmospheric_light = np.max(image.reshape(-1, 3)[indices], axis=0)
    return atmospheric_light

In [16]:
def transmission_estimation(image, atmospheric_light, omega=0.95, window_size=15):
    normalized_image = image / atmospheric_light
    dark_chan = dark_channel(normalized_image, window_size)
    transmission_map = 1.0 - omega * dark_chan
    return transmission_map


In [17]:
def guided_filter(I, p, r=15, eps=1e-3):
    mean_I  = cv2.boxFilter(I, -1, (r, r))
    mean_p  = cv2.boxFilter(p, -1, (r, r))
    corr_I  = cv2.boxFilter(I * I, -1, (r, r))
    corr_Ip = cv2.boxFilter(I * p, -1, (r, r))

    var_I = corr_I - mean_I * mean_I
    cov_Ip = corr_Ip - mean_I * mean_p

    a = cov_Ip / (var_I + eps)
    b = mean_p - a * mean_I

    mean_a = cv2.boxFilter(a, -1, (r, r))
    mean_b = cv2.boxFilter(b, -1, (r, r))

    q = mean_a * I + mean_b
    return q

In [18]:
def dehaze(image, omega=0.95, window_size=15, t_min=0.1):
    dark_chan = dark_channel(image, window_size)
    atmospheric = atmospheric_light(image, dark_chan)
    transmission = transmission_estimation(image, atmospheric, omega, window_size)
    transmission = np.maximum(transmission, t_min)
    
    # Resize the transmission map to match the shape of the input image
    transmission_resized = cv2.resize(transmission, (image.shape[1], image.shape[0]))
    
    # Expand dimensions of the transmission map to match number of channels in the input image
    transmission_expanded = np.repeat(transmission_resized[:, :, np.newaxis], 3, axis=2)
    
    refined_transmission = guided_filter(image.astype(np.float32)/255.0 , transmission_expanded.astype(np.float32))

    recovered_image = np.zeros_like(image)
    
    for i in range(3):
        recovered_image[:, :, i] = ((image[:, :, i] - atmospheric[i]) / refined_transmission[:, :, i] + atmospheric[i]).astype(np.uint8)

    return np.clip(recovered_image.astype(np.uint8), 0 ,255)


In [19]:
# Get the current directory
current_directory = os.getcwd()

# Define input and output folder paths
input_folder = os.path.join(current_directory, 'Input')
output_folder = os.path.join(current_directory, 'Output')

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# List all image files in the input folder
image_files = [f for f in os.listdir(input_folder) if f.endswith(('.jpg', '.jpeg', '.png', '.bmp'))]

for image_file in image_files:
    # Load the underwater image
    input_image_path = os.path.join(input_folder, image_file)
    input_image = cv2.imread(input_image_path)

    # Enhance the image using the Dark Channel Prior
    enhanced_image = dehaze(input_image)

    # Save the enhanced image to the output folder within the input folder
    output_image_path = os.path.join(output_folder, image_file)
    cv2.imwrite(output_image_path, enhanced_image)

print("Enhancement complete. Enhanced images are saved in the 'Output' folder within the 'Input' folder.")

Enhancement complete. Enhanced images are saved in the 'Output' folder within the 'Input' folder.
